# English to Spanish Fine Tuning

Starting from the Hugging Face Translation Tutorial (https://huggingface.co/learn/nlp-course/en/chapter7/4#preparing-the-data)

In [89]:
from tqdm import tqdm
import numpy as np

# hugging face translation tutorial
from datasets import load_dataset
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
import evaluate
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
# custom training loop
from torch.utils.data import DataLoader
from torch.optim import AdamW
from accelerate import Accelerator
from transformers import get_scheduler
from tqdm.auto import tqdm

# customize model
from transformers import AutoModel, PretrainedConfig, AutoConfig

# torch
import torch
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

device

'cpu'

## Notes

### 2025-03-21
* May be interested in using Subword Segmentation on rare/novel words?
* May be interested in using a different dataset than `helsinski-NLP/kde4`
  * Weird errors like english words attached to end of spanish translations
  * Small sentences, maybe this was a textbook?

**Tip**: Restart the kernel every time you have to download something...

Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [2]:
#pip install tf-keras --user

In [3]:
#pip install transformers

In [ ]:
#!pip install transformers[torch] OR !pip install 'accelerate>=0.26.0

In [4]:
#!pip install datasets

In [5]:
#!pip install evaluate

In [6]:
#!pip install sentencepiece

In [7]:
#!pip install sacrebleu

## Load Dataset(s)

In [90]:
# use Helinski-NLP/kde4 dataset (https://huggingface.co/datasets/Helsinki-NLP/kde4)
# this dataset has a good amount of oddness/errors, we should probably consider something else
raw_datasets = load_dataset("kde4", lang1="en", lang2="es", trust_remote_code=True)

In [91]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 218655
    })
})

In [92]:
# split again for validation set
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

In [93]:
split_datasets["train"][84]["translation"]

{'en': 'Left Arrow', 'es': 'Tecla de dirección izquierda.'}

In [94]:
split_datasets["train"][22]["translation"] # interesting...

{'en': 'Edit and paint images', 'es': 'Editar y pintar imágenesName'}

## Process Data

In [95]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-es"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [96]:
split_datasets["train"][66]["translation"]

{'en': 'Removes the selected render mode.',
 'es': 'Elimina el modo de procesado seleccionado.'}

In [97]:
en_sentence = split_datasets["train"][66]["translation"]["en"]
es_sentence = split_datasets["train"][66]["translation"]["es"]

inputs = tokenizer(en_sentence, text_target=es_sentence)
inputs

{'input_ids': [23292, 9, 5, 4836, 17058, 9888, 3, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1], 'labels': [47448, 14, 1160, 4, 38117, 15569, 3, 0]}

In [98]:
wrong_targets = tokenizer(es_sentence) # english tokenizer preprocessing spanish sentence, does not do well (this is demo stuff, just to make sure the tokenizer is applied correctly)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁Eli', 'mina', '▁el', '▁mod', 'o', '▁de', '▁pro', 'ces', 'ado', '▁s', 'ele', 'c', 'cion', 'ado', '.', '</s>']
['▁Elimina', '▁el', '▁modo', '▁de', '▁procesado', '▁seleccionado', '.', '</s>']


In [99]:
max_length = 128 # depends on how long text is... text does not seeem to be very long?

def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["es"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_length, truncation=True)
    return model_inputs

⚠️ We don’t pay attention to the attention mask of the targets, as the model won’t expect it. Instead, the labels corresponding to a padding token should be set to -100 so they are ignored in the loss computation. This will be done by our data collator later on since we are applying dynamic padding, but if you use padding here, you should adapt the preprocessing function to set all labels that correspond to the padding token to -100.

In [100]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

# Model

In [101]:
# demo
model_checkpoint = "Helsinki-NLP/opus-mt-en-es"
translator = pipeline("translation", model=model_checkpoint)
translator("Have you considered horse?")

Device set to use cpu


[{'translation_text': '¿Has considerado el caballo?'}]

In [102]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

### Data Collation
Deals with padding for dynamic batching. Need to pad labels to maximum length.

I do not know what this means beyond that information.

In [103]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [104]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [105]:
batch["labels"]

tensor([[ 3244,   755,     0,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100],
        [  664,  1532, 12100,    19, 17220,  6533,  2834,  1257, 15569,     3,
             0]])

In [106]:
batch["decoder_input_ids"]

tensor([[65000,  3244,   755,     0, 65000, 65000, 65000, 65000, 65000, 65000,
         65000],
        [65000,   664,  1532, 12100,    19, 17220,  6533,  2834,  1257, 15569,
             3]])

In [107]:
# labels for first and second elements in dataset
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[3244, 755, 0]
[664, 1532, 12100, 19, 17220, 6533, 2834, 1257, 15569, 3, 0]


### Metrics

In [108]:
metric = evaluate.load("sacrebleu")

In [109]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits :eyes:
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them (these are padding labels)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

### Fine Tuning - Custom Training Loop

In [110]:
tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [111]:
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [112]:
class MODEL(AutoModelForSeq2SeqLM): # not sure if this is the correct type, but here we go # AutoModelForSeq2SeqLM
    def __init__(self, config):
        super().__init__(config) # I'm not sure if this,,, creates another set of layers on top?
        self.l1 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

#         self.pre_classifier = torch.nn.Linear(768, 256)
        # self.classifier = torch.nn.Linear(768, NUM_OUT)
#         self.dropout = torch.nn.Dropout(0.5)
        # self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.l1.forward(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
#         hidden_state = output_1[0]
#         pooler = hidden_state[:, 0]
# #         pooler = self.pre_classifier(pooler)
# #         pooler = torch.nn.Tanh()(pooler)
# #         pooler = self.dropout(pooler)
#         output = self.classifier(pooler)
#         output = self.softmax(output)
        # return output

        # output = self.electra(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        return output #.logits

    def get_logits(self, input_ids, attention_mask, token_type_ids):
        output = self.l1.forward(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        return output.logits

In [113]:
class CONFIG(PretrainedConfig):
    model_type = "our-model"

    def __init__(
        self,
        vocab_size=30522,
        d_model=512,
        d_kv=64,
        d_ff=2048,
        num_layers=6,
        num_decoder_layers=6,
        num_heads=8,
        relative_attention_num_buckets=32,
        relative_attention_max_distance=128,
        dropout_rate=0.1,
        layer_norm_epsilon=1e-6,
        initializer_factor=1.0,
        feed_forward_proj="relu",
        use_cache=True,
        pad_token_id=0,
        eos_token_id=1,
        **kwargs,
    ):
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.d_kv = d_kv
        self.d_ff = d_ff
        self.num_layers = num_layers
        self.num_decoder_layers = num_decoder_layers
        self.num_heads = num_heads
        self.relative_attention_num_buckets = relative_attention_num_buckets
        self.relative_attention_max_distance = relative_attention_max_distance
        self.dropout_rate = dropout_rate
        self.layer_norm_epsilon = layer_norm_epsilon
        self.initializer_factor = initializer_factor
        self.feed_forward_proj = feed_forward_proj
        self.use_cache = use_cache

        super().__init__(pad_token_id=pad_token_id, eos_token_id=eos_token_id, **kwargs)
    

In [114]:
AutoConfig.register("our-model", CONFIG)
AutoModel.register(CONFIG, MODEL)

config = AutoConfig.from_pretrained(model_checkpoint)
model = MODEL.from_config(config)

In [115]:
optimizer = AdamW(model.parameters(), lr=2e-5)

In [116]:
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [117]:
num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [118]:
# push to huggingface hubm (needs login)
# from huggingface_hub import Repository, get_full_repo_name

# model_name = "marian-finetuned-kde4-en-to-es-accelerate"
# repo_name = get_full_repo_name(model_name)
# output_dir = "marian-finetuned-kde4-en-to-es-accelerate"
# repo = Repository(output_dir, clone_from=repo_name)

In [119]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    # accelerator.wait_for_everyone()
    # unwrapped_model = accelerator.unwrap_model(model)
    # unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    # if accelerator.is_main_process:
    #     tokenizer.save_pretrained(output_dir)
    #     repo.push_to_hub(
    #         commit_message=f"Training in progress epoch {epoch}", blocking=False
    #     )

  0%|          | 0/73797 [00:00<?, ?it/s]

### Fine Tuning - Pre Defined Loop

Log into hugging face here:
```py
from huggingface_hub import notebook_login
notebook_login()
```
OR
```sh
huggingface-cli login
```

In [34]:
args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-es",
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True, # speed up training on modern GPUs
    push_to_hub=False, # I don't know how to do this
)

In [37]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [39]:
# score original model as baseline to ensure fine tuning actually improves
trainer.evaluate(max_length=max_length) # takes time

KeyboardInterrupt: 

In [ ]:
trainer.train() # takes time

In [ ]:
trainer.evaluate(max_length=max_length) # takes time

### Demo with Fine Tuned Model

In [ ]:
# Replace this with your own checkpoint
model_checkpoint = "huggingface-course/marian-finetuned-kde4-en-to-es"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

In [ ]:
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)

### Tokenizer Checking

In [25]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

In [26]:
tokenizer

NllbTokenizerFast(name_or_path='facebook/nllb-200-distilled-600M', vocab_size=256204, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['ace_Arab', 'ace_Latn', 'acm_Arab', 'acq_Arab', 'aeb_Arab', 'afr_Latn', 'ajp_Arab', 'aka_Latn', 'amh_Ethi', 'apc_Arab', 'arb_Arab', 'ars_Arab', 'ary_Arab', 'arz_Arab', 'asm_Beng', 'ast_Latn', 'awa_Deva', 'ayr_Latn', 'azb_Arab', 'azj_Latn', 'bak_Cyrl', 'bam_Latn', 'ban_Latn', 'bel_Cyrl', 'bem_Latn', 'ben_Beng', 'bho_Deva', 'bjn_Arab', 'bjn_Latn', 'bod_Tibt', 'bos_Latn', 'bug_Latn', 'bul_Cyrl', 'cat_Latn', 'ceb_Latn', 'ces_Latn', 'cjk_Latn', 'ckb_Arab', 'crh_Latn', 'cym_Latn', 'dan_Latn', 'deu_Latn', 'dik_Latn', 'dyu_Latn', 'dzo_Tibt', 'ell_Grek', 'eng_Latn', 'epo_Latn', 'est_Latn', 'eus_Latn', 'ewe_Latn', 'fao_Latn', 'pes_Arab'

In [27]:
en_sentence = split_datasets["train"][66]["translation"]["en"]
es_sentence = split_datasets["train"][66]["translation"]["es"]

inputs = tokenizer(en_sentence, text_target=es_sentence)
inputs

TypeError: 'NoneType' object is not iterable

In [28]:
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁diɲɛ', 'er', '▁से', 'an', '▁ஐ', 'ski', '<unk>', '<s>']


### Attempt at logits

In [126]:
model.eval()
evaluated_samples = 0
num_samples = 25
all_logits = []
for batch in tqdm(eval_dataloader):
    if evaluated_samples >= num_samples:
        break
    
    with torch.no_grad():
        generated_tokens = accelerator.unwrap_model(model).generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
            max_length=128,
        )
        logits = outputs.logits
        all_logits.append(logits)
    labels = batch["labels"]

    # Necessary to pad predictions and labels for being gathered
    generated_tokens = accelerator.pad_across_processes(
        generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
    )
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    predictions_gathered = accelerator.gather(generated_tokens)
    labels_gathered = accelerator.gather(labels)

    decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
    metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    evaluated_samples += len(batch["input_ids"])

results = metric.compute()
print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

all_logits = torch.cat(all_logits, dim=0)
print(all_logits.shape)

  0%|          | 0/2734 [00:00<?, ?it/s]

epoch 0, BLEU score: 0.00
torch.Size([32, 110, 65001])


In [127]:
all_logits

tensor([[[ 6.4986e+00,  5.7906e-02,  2.7258e+00,  ..., -1.1224e+00,
          -9.5362e-01, -1.4607e+00],
         [ 6.5390e+00, -7.7212e-02,  2.7722e+00,  ..., -1.0668e+00,
          -9.4923e-01, -1.4589e+00],
         [ 6.6249e+00,  8.3936e-02,  2.7289e+00,  ..., -1.0488e+00,
          -9.2998e-01, -1.4554e+00],
         ...,
         [ 6.7025e+00,  1.2496e-01,  2.7672e+00,  ..., -1.2927e+00,
          -8.6860e-01, -1.4546e+00],
         [ 6.7182e+00,  1.0310e-01,  2.8779e+00,  ..., -1.1571e+00,
          -9.8894e-01, -1.4538e+00],
         [ 6.7133e+00,  4.0660e-02,  2.8156e+00,  ..., -1.0568e+00,
          -9.8556e-01, -1.4508e+00]],

        [[ 6.4081e+00, -1.8062e-02,  2.7849e+00,  ..., -1.0841e+00,
          -9.3041e-01, -1.4581e+00],
         [ 6.4542e+00,  7.2807e-02,  2.7681e+00,  ..., -1.2727e+00,
          -1.1708e+00, -1.4649e+00],
         [ 6.5058e+00, -1.0938e-02,  2.8437e+00,  ..., -1.0158e+00,
          -9.3117e-01, -1.4612e+00],
         ...,
         [ 6.5045e+00,  6

## Remnants of BERT

In [42]:
X = [line.strip() for line in open('X.txt').readlines()]
y = train_data = [int(line.strip()) for line in open('YL1.txt').readlines()]

len(X), len(y), max(y)

(46985, 46985, 6)

In [43]:
train_X = X[:46000]
train_y = np.array(y[:46000])
test_X = X[46000:]
test_y = np.array(y[46000:])

len(train_X), len(train_y), len(test_X), len(test_y)

(46000, 46000, 985, 985)

In [44]:
# not needed for training or evaluation, but useful for mapping examples
labels = {
    0:'Computer Science',
    1:'Electrical Engineering',
    2:'Psychology',
    3:'Mechanical Engineering',
    4:'Civil Engineering',
    5:'Medical Science',
    6:'Biochemistry'
}

len(labels)

7

#### Torch Datasets

* takes in inputs and outputs/labels
* interfaces with tokenizer
* handles batching

In [45]:
class MultiLabelDataset(torch.utils.data.Dataset):

    def __init__(self, text, labels, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = text
        self.targets = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            # 'targets': torch.tensor(self.targets[index], dtype=torch.long) # was float
            'targets': self.targets[index].clone().detach()
        }

#### BERT Class

* first layer is pretrained BERT model
* add whatever layers after

In [10]:
class ELECTRAClass(torch.nn.Module):
    def __init__(self, NUM_OUT):
        # super(BERTClass, self).__init__()
        super(ELECTRAClass, self).__init__()
        # self.l1 = BertModel.from_pretrained("bert-base-uncased")
        self.electra = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=NUM_OUT)

#         self.pre_classifier = torch.nn.Linear(768, 256)
        # self.classifier = torch.nn.Linear(768, NUM_OUT)
#         self.dropout = torch.nn.Dropout(0.5)
        # self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
#         output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
#         hidden_state = output_1[0]
#         pooler = hidden_state[:, 0]
# #         pooler = self.pre_classifier(pooler)
# #         pooler = torch.nn.Tanh()(pooler)
# #         pooler = self.dropout(pooler)
#         output = self.classifier(pooler)
#         output = self.softmax(output)
        # return output

        output = self.electra(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        return output.logits

#### Helpful Functions

Loss
* Tasks with only two labels use binary crossentropy
* Tasks with more labels will use categorical crossentropy
* Tasks that don't have labels, but rather have distributions should use KL divergence
* Tasks that don't have distributions should use something like RMSE loss

Train
* Steps through the data batch by batch
* grabs ids, masks, and token_type_ids which are required inputs for BERT
* inputs are passed through the model, compared to targets, computes loss function, backprops

Validation
* Takes a model, passes inputs
* Need to use the targets from here because they are potentially shuffled!

In [50]:
def loss_fn(outputs, targets):
    # return torch.nn.BCELoss()(outputs, targets)
    return torch.nn.CrossEntropyLoss()(outputs, targets) # not likely to work on first go

In [51]:
def train(model, training_loader, optimizer):
    model.train()
    for data in tqdm(training_loader):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long) # was float

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss

In [52]:
def validation(model, testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for data in tqdm(testing_loader):
            targets = data['targets'].to(device, dtype=torch.long)
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids)
            outputs = torch.argmax(outputs, dim=1).cpu().detach()
            # outputs = torch.sigmoid(outputs).cpu().detach()
            fin_outputs.extend(outputs)
            fin_targets.extend(targets)
    return torch.stack(fin_outputs), torch.stack(fin_targets)

#### The Tokenizer

* converts raw string to ids, masks, and token_type_ids

In [14]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = ElectraTokenizerFast.from_pretrained('google/electra-small-discriminator')

# what does the tokenizer do?
print(f"train data: {train_X[5]}")

tokenizer.encode_plus(
            train_X[5],
            None,
            add_special_tokens=True,
            max_length=128,
            # pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

train data: (Objective) In order to increase classification accuracy of tea-category identification (TCI) system, this paper proposed a novel approach. (Method) The proposed methods first extracted 64 color histogram to obtain color information, and 16 wavelet packet entropy to obtain the texture information. With the aim of reducing the 80 features, principal component analysis was harnessed. The reduced features were used as input to generalized eigenvalue proximal support vector machine (GEPSVM). Winner-takes-all (WTA) was used to handle the multiclass problem. Two kernels were tested, linear kernel and Radial basis function (RBF) kernel. Ten repetitions of 10-fold stratified cross validation technique were used to estimate the out-of-sample errors. We named our method as GEPSVM + RBF + WTA and GEPSVM + WTA. (Result) The results showed that PCA reduced the 80 features to merely five with explaining 99.90% of total variance. The recall rate of GEPSVM + RBF + WTA achieved the highest 

{'input_ids': [101, 1006, 7863, 1007, 1999, 2344, 2000, 3623, 5579, 10640, 1997, 5572, 1011, 4696, 8720, 1006, 22975, 2072, 1007, 2291, 1010, 2023, 3259, 3818, 1037, 3117, 3921, 1012, 1006, 4118, 1007, 1996, 3818, 4725, 2034, 15901, 4185, 3609, 2010, 3406, 13113, 2000, 6855, 3609, 2592, 1010, 1998, 2385, 4400, 7485, 14771, 23077, 2000, 6855, 1996, 14902, 2592, 1012, 2007, 1996, 6614, 1997, 8161, 1996, 3770, 2838, 1010, 4054, 6922, 4106, 2001, 17445, 2098, 1012, 1996, 4359, 2838, 2020, 2109, 2004, 7953, 2000, 18960, 1041, 29206, 10175, 5657, 4013, 9048, 9067, 2490, 9207, 3698, 1006, 16216, 4523, 2615, 2213, 1007, 1012, 3453, 1011, 3138, 1011, 2035, 1006, 21925, 1007, 2001, 2109, 2000, 5047, 1996, 4800, 26266, 3291, 1012, 2048, 16293, 2015, 2020, 7718, 1010, 7399, 16293, 1998, 15255, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

#### Training setup

* hyperparameters
* setup dataset
* setup parameters
* setup dataloader

In [53]:
MAX_LEN = 128
BATCH_SIZE = 64 #original: 64
EPOCHS = 3
NUM_OUT = len(labels)
LEARNING_RATE = 2e-05

training_data = MultiLabelDataset(train_X, torch.from_numpy(train_y), tokenizer, MAX_LEN)
test_data = MultiLabelDataset(test_X, torch.from_numpy(test_y), tokenizer, MAX_LEN)

train_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

test_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

training_loader = torch.utils.data.DataLoader(training_data, **train_params)
testing_loader = torch.utils.data.DataLoader(test_data, **test_params)

#### Train, Evaluate

* model.to -> send to GPU, if available (anything computed should be put onto the GPU)
* setup optimizer - could use Stochastic Gradient Descent, but ADAM tends to work better
* for each epoch, train, show the loss, evaluate on the test data

In [16]:
# model = BERTClass(NUM_OUT)
model = ELECTRAClass(NUM_OUT)
model.to(device)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    loss = train(model, training_loader, optimizer)
    print(f'Epoch: {epoch}, Loss:  {loss.item()}')
    guess, targs = validation(model, testing_loader)
    # guesses = torch.max(guess, dim=1)
    guesses = guess.cpu()
    # targets = torch.max(targs, dim=1)
    targets = targs.cpu()
    # print('arracy on test set {}'.format(accuracy_score(guesses.indices, targets.indices)))
    print("\naccuracy on test set {}".format(accuracy_score(guesses, targets)))

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/719 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transforme

model.safetensors:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

100%|██████████| 719/719 [05:47<00:00,  2.07it/s]


Epoch: 0, Loss:  0.889681339263916


...


accuracy on test set 0.7857868020304568


  0%|          | 0/719 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_leng

Epoch: 2, Loss:  0.5695027709007263


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_lengt


accuracy on test set 0.8060913705583757


In [ ]:
# terminate this colab session to release resources
# from google.colab import runtime
# runtime.unassign()